Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en preguntas de código y preguntas teóricas, con la puntuación que se indica a continuación. La puntuación máxima será 10.

- [Actividad 1: Redes Densas](#actividad_1): 10 pts
    - Correcta normalización: máximo de 0.5 pts
    - [Cuestión 1](#1.1): 1.5 pts
    - [Cuestión 2](#1.2): 1.5 pts
    - [Cuestión 3](#1.3): 1.5 pts
    - [Cuestión 4](#1.4): 1 pts
    - [Cuestión 5](#1.5): 1 pts
    - [Cuestión 6](#1.6): 1 pts
    - [Cuestión 7](#1.7): 1 pts
    - [Cuestión 8](#1.8): 1 pts


In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

<a name='actividad_1'></a>
# Actividad 1: Redes Densas

Para esta actividad vamos a utilizar el [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality). Con el que trataremos de predecir la calidad del vino.

La calidad del vino puede tomar valores decimales (por ejemplo 7.25), independientemente de que en el dataset de entrenamiento sean números enteros. Por lo tanto, el problema es una `regresión`.

**Puntuación**:

Normalizar las features correctamente (x_train, x_test): 0.5 pts

- Correcta normalización: máximo de 0.5 pts
- [Cuestión 1](#1.1): 1 pt
- [Cuestión 2](#1.2): 1 pt
- [Cuestión 3](#1.3): 0.5 pts
- [Cuestión 4](#1.4): 0.5 pts
- [Cuestión 5](#1.5): 0.5 pts
- [Cuestión 6](#1.6): 0.5 pts
- [Cuestión 7](#1.7): 0.5 pts
- [Cuestión 8](#1.8): 0.5 pts



In [62]:
# Descargar los datos con pandas
df_red = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
    sep=';'
)
df_white = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
    sep=';'
)
df = pd.concat([df_red, df_white])

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [63]:
feature_names = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
]


# separar features y target
y = df.pop('quality').values
X = df.copy().values

In [64]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)
print('Some qualities: ', y_train[:5])

x_train, y_train shapes: (4872, 11) (4872,)
x_test, y_test shapes: (1625, 11) (1625,)
Some qualities:  [6 7 8 5 6]


In [65]:
## Si quiere, puede normalizar las features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print('X_train mu, sigma', x_train.mean(0), x_train.std(0))
print('X_test mu, sigma', x_test.mean(0), x_test.std(0))

X_train mu, sigma [ 1.15064863e-15  1.43563322e-15  2.53269400e-14  3.71760641e-15
 -3.57372975e-16 -1.09096731e-16 -5.61862407e-18  1.85223021e-12
 -3.61682951e-14  5.10059975e-15 -4.98973924e-14] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X_test mu, sigma [ 0.00529427 -0.0206874  -0.01685718  0.06516557 -0.0197353   0.00200499
 -0.00445979  0.04445494  0.03272033 -0.02514518 -0.01054552] [1.01770688 0.97832455 0.99432368 1.09585347 0.94447751 0.95418223
 0.98973067 1.08890349 1.00225041 0.94519387 1.01278965]


<a name='1.1'></a>
## Cuestión 1: Cree un modelo secuencial que contenga 4 capas ocultas(hidden layers), con más de 60 neuronas  por capa, sin regularización y obtenga los resultados.

Puntuación:
- Obtener el modelo correcto: 0.8 pts
- Compilar el modelo: 0.1pts
- Acertar con la función de pérdida: 0.1 pts

In [66]:
model = tf.keras.models.Sequential()

model.add(layers.Dense(256, input_shape=(11,), activation='relu'))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

# output layer
model.add(layers.Dense(1))

# model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_41 (Dense)                │ (None, 256)            │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,801 (237.50 KB)

 Trainable params: 60,801 (237.50 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Compilación del modelo
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MSE,
    metrics=['mae']
)

In [68]:
# No modifique el código
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 10.2262 - mae: 2.4456 - val_loss: 1.5827 - val_mae: 0.9007
Epoch 2/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.1692 - mae: 0.8417 - val_loss: 0.8709 - val_mae: 0.7125
Epoch 3/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6837 - mae: 0.6513 - val_loss: 0.5908 - val_mae: 0.5853
Epoch 4/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5527 - mae: 0.5720 - val_loss: 0.5772 - val_mae: 0.5877
Epoch 5/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4955 - mae: 0.5480 - val_loss: 0.5673 - val_mae: 0.5818
Epoch 6/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5062 - mae: 0.5580 - val_loss: 0.5252 - val_mae: 0.5532
Epoch 7/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4738 - mae: 0.5338 - val_loss: 0.5846 - val_mae: 0.5867
Epoch 8/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4861 - mae: 0.5396 - val_loss: 0.5291 - val_mae: 0.5619
Epoch 9/200
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5m

In [69]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5704 - mae: 0.5365
Test Loss: [0.5960760116577148, 0.5437871217727661]


<a name='1.2'></a>
## Cuestión 2: Utilice el mismo modelo de la cuestión anterior pero añadiendo al menos dos técnicas distinas de regularización. No es necesario reducir el test loss.

Ejemplos de regularización: [Prevent_Overfitting.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/Prevent_Overfitting.ipynb)

In [81]:
kernel_regularizer_l2 = keras.regularizers.l2(5e-4)

model = tf.keras.models.Sequential()

model.add(layers.Dense(256, input_shape=(11,), activation='relu'))

model.add(layers.Dropout(0.4))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(128, activation='relu', kernel_regularizer=kernel_regularizer_l2))

model.add(layers.Dropout(0.4))

model.add(layers.Dense(64, activation='relu'))

# output layer
model.add(layers.Dense(1))

# model summary
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_61 (Dense)                │ (None, 256)            │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,801 (237.50 KB)

 Trainable params: 60,801 (237.50 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
# Compilación del modelo
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MSE,
    metrics=['mae']
)

In [83]:
batch_size=64

In [84]:
# No modifique el código
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=batch_size,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 14.3475 - mae: 3.1756 - val_loss: 2.1839 - val_mae: 1.0734
Epoch 2/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.3989 - mae: 1.1919 - val_loss: 1.8020 - val_mae: 1.0425
Epoch 3/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.7904 - mae: 1.0413 - val_loss: 1.2595 - val_mae: 0.8420
Epoch 4/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4222 - mae: 0.9154 - val_loss: 1.5772 - val_mae: 1.0329
Epoch 5/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2322 - mae: 0.8488 - val_loss: 0.9481 - val_mae: 0.7420
Epoch 6/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0550 - mae: 0.7917 - val_loss: 1.0168 - val_mae: 0.7825
Epoch 7/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9669 - mae: 0.7530 - val_loss: 0.9294 - val_mae: 0.7429
Epoch 8/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9313 - mae: 0.7301 - val_loss: 1.1352 - val_mae: 0.8612
Epoch 9/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8

In [85]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.4677 - mae: 0.4949
Test Loss: [0.4906873404979706, 0.505670428276062]


<a name='1.3'></a>
## Cuestión 3: Utilice el mismo modelo de la cuestión anterior pero añadiendo un callback de early stopping. No es necesario reducir el test loss.

In [ ]:
kernel_regularizer_l2 = keras.regularizers.l2(5e-4)

model = tf.keras.models.Sequential()

model.add(layers.Dense(256, input_shape=(11,), activation='relu'))

model.add(layers.Dropout(0.4))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(128, activation='relu', kernel_regularizer=kernel_regularizer_l2))

model.add(layers.Dropout(0.4))

model.add(layers.Dense(64, activation='relu'))

# output layer
model.add(layers.Dense(1))

# model summary
model.summary()

In [ ]:
# Compilación del modelo
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MSE,
    metrics=['mae']
)

In [ ]:
## definir el early stopping callback
# Código aquí
...
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[...]) # Código aquí

In [ ]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

<a name='1.4'></a>
## Cuestión 4: ¿Podría haberse usado otra función de activación de la neurona de salida? En caso afirmativo especifíquela.

<a name='1.5'></a>
## Cuestión 5:  ¿Qué es lo que una neurona calcula?

**a)** Una función de activación seguida de una suma ponderada  de las entradas.

**b)** Una suma ponderada  de las entradas seguida de una función de activación.

**c)** Una función de pérdida, definida sobre el target.

**d)** Ninguna  de las anteriores es correcta


<a name='1.6'></a>
## Cuestión 6:  ¿Cuál de estas funciones de activación no debería usarse en una capa oculta (hidden layer)?

**a)** `sigmoid`

**b)** `tanh`

**c)** `relu`

**d)** `linear`


<a name='1.7'></a>
## Cuestión 7:  ¿Cuál de estas técnicas es efectiva para combatir el overfitting en una red con varias capas ocultas? Ponga todas las que lo sean.

**a)** Dropout

**b)** Regularización L2.

**c)** Aumentar el tamaño del test set.

**d)** Aumentar el tamaño del validation set.

**e)** Reducir el número de capas de la red.

**f)** Data augmentation.

<a name='1.8'></a>
## Cuestión 8:  Supongamos que queremos entrenar una red para un problema de clasificación de imágenes con las siguientes clases: {'perro','gato','persona'}. ¿Cuántas neuronas y que función de activación debería tener la capa de salida? ¿Qué función de pérdida (loss function) debería usarse?
